# BERT: Contextual Word Embeddings

BERT (Bidirectional Encoder Representations from Transformers) revolutionized NLP by introducing **contextual embeddings** — the same word gets different vector representations depending on its surrounding context.

**Key differences from Word2Vec:**
- **Word2Vec**: Static embeddings — "bank" always has the same vector
- **BERT**: Contextual embeddings — "river bank" vs "bank account" produce different vectors for "bank"

**Key concepts:**
- **Transformer architecture**: Self-attention mechanism that considers all words simultaneously
- **Bidirectional**: Considers both left and right context (unlike GPT which is left-to-right only)
- **WordPiece tokenization**: Subword units handle unknown words gracefully
- **[CLS] token**: Special token whose embedding represents the entire sequence

---

## 1. Setup & Installation

In [1]:
!pip install transformers torch sentence-transformers scipy -q

In [2]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
import numpy as np

/Users/brianchew/dev/is469-genaiwithllms/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load pre-trained BERT model and tokenizer
# bert-base-uncased: 12 layers, 768 hidden size, 110M parameters
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set to evaluation mode (disables dropout)

print(f"Model config: {model.config.num_hidden_layers} layers, {model.config.hidden_size} hidden size")

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1016.86it/s, Materializing param=pooler.dense.weight]                              
BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model config: 12 layers, 768 hidden size


### Alternative: Using AutoTokenizer (Class Style)

Your class uses `AutoTokenizer` and `AutoModel` which auto-detect the right class. Also uses `bert-base-cased` (preserves case).

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Class uses bert-base-cased (case-sensitive: "Hello" ≠ "hello")
model_name = "bert-base-cased"
auto_tokenizer = AutoTokenizer.from_pretrained(model_name)
auto_model = AutoModel.from_pretrained(model_name)

print(f"Loaded: {model_name}")

In [ ]:
# Tokenize text (class style)
text = "Hello"
tokens = auto_tokenizer(text)

print("Token dict:", tokens)
print("Input IDs:", tokens['input_ids'])  # [101, 8667, 102] = [CLS], Hello, [SEP]

### Accessing the Embedding Layer Directly

BERT's embedding layer is a lookup table mapping token IDs → 768-dim vectors. This is the **static** part before contextualization happens in the transformer layers.

In [ ]:
# Access the embedding layer (like in class)
embedding_layer = auto_model.embeddings
word_embeddings = embedding_layer.word_embeddings

# The weight matrix: [vocab_size, hidden_size]
print("Embedding weight shape:", word_embeddings.weight.shape)
# bert-base-cased has 28,996 tokens, each with 768 dimensions

print("\nFirst 5 rows of embedding matrix:")
print(word_embeddings.weight[:5])

In [ ]:
# Get embedding for a specific token by ID
# "Hello" has token_id 8667 in bert-base-cased
token_id = auto_tokenizer.encode("Hello", add_special_tokens=False)[0]
print(f"Token ID for 'Hello': {token_id}")

# Look up its embedding directly from the weight matrix
hello_embedding = word_embeddings.weight[token_id]
print(f"Embedding shape: {hello_embedding.shape}")
print(f"First 10 dims: {hello_embedding[:10]}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare embeddings from the embedding layer (BEFORE contextualization)
def get_token_embedding(word):
    """Get the raw embedding (non-contextual) for a word."""
    token_ids = auto_tokenizer.encode(word, add_special_tokens=False)
    if len(token_ids) > 1:
        print(f"Warning: '{word}' splits into {len(token_ids)} tokens")
    emb = word_embeddings.weight[token_ids[0]].detach().numpy()
    return emb.reshape(1, -1)

# Compare some word pairs
word_pairs = [
    ("Hello", "Hi"),
    ("king", "queen"),
    ("cat", "dog"),
    ("cat", "computer"),
]

print("Cosine similarity (raw embeddings, non-contextual):\n")
for w1, w2 in word_pairs:
    emb1 = get_token_embedding(w1)
    emb2 = get_token_embedding(w2)
    sim = cosine_similarity(emb1, emb2)[0][0]
    print(f"  {w1} <-> {w2}: {sim:.4f}")

**Note:** The embeddings above are from the **embedding layer** (static, like Word2Vec). The real power of BERT comes from passing these through 12 transformer layers to get **contextual** embeddings (shown in section 3).

## 2. BERT Tokenization (WordPiece)

BERT uses **WordPiece** tokenization:
- Common words stay intact: "the", "cat", "running"
- Rare/unknown words split into subwords: "embeddings" → "em", "##bed", "##ding", "##s"
- `##` prefix indicates a continuation of the previous token

Special tokens:
- `[CLS]`: Added at the start, its embedding represents the whole sequence
- `[SEP]`: Separates sentences (used in sentence-pair tasks)
- `[PAD]`: Padding for batch processing
- `[MASK]`: Used during pre-training (masked language modeling)

In [4]:
# Basic tokenization example
text = "The cat sat on the mat."

# Tokenize into subwords
tokens = tokenizer.tokenize(text)
print(f"Original: {text}")
print(f"Tokens: {tokens}")

# Convert to token IDs (what the model actually sees)
token_ids = tokenizer.encode(text)
print(f"Token IDs: {token_ids}")

# Decode back to text
decoded = tokenizer.decode(token_ids)
print(f"Decoded: {decoded}")

Original: The cat sat on the mat.
Tokens: ['the', 'cat', 'sat', 'on', 'the', 'mat', '.']
Token IDs: [101, 1996, 4937, 2938, 2006, 1996, 13523, 1012, 102]
Decoded: [CLS] the cat sat on the mat. [SEP]


In [5]:
# Subword splitting for complex/rare words
examples = [
    "embeddings",           # Technical term
    "unbelievable",         # Long word with prefixes
    "transformers",         # Domain-specific
    "antidisestablishment", # Very long word
    "ChatGPT",              # Modern term (not in BERT's vocab)
]

for word in examples:
    tokens = tokenizer.tokenize(word)
    print(f"{word:25} → {tokens}")

embeddings                → ['em', '##bed', '##ding', '##s']
unbelievable              → ['unbelievable']
transformers              → ['transformers']
antidisestablishment      → ['anti', '##dis', '##est', '##ab', '##lish', '##ment']
ChatGPT                   → ['chat', '##gp', '##t']


In [6]:
# Full tokenization with special tokens
text = "Hello, how are you?"

# Using the tokenizer to get model inputs
inputs = tokenizer(text, return_tensors='pt')

print("Input IDs:", inputs['input_ids'])
print("Attention mask:", inputs['attention_mask'])
print("Tokens:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Note: [CLS] at start (101), [SEP] at end (102)

Input IDs: tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]])
Attention mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
Tokens: ['[CLS]', 'hello', ',', 'how', 'are', 'you', '?', '[SEP]']


## 3. Extracting Word Embeddings

BERT outputs a 768-dimensional vector for each token. The key insight: **the same word gets different embeddings based on context**.

The output shape is `[batch_size, sequence_length, hidden_size]` where:
- `batch_size`: Number of sentences (1 for single sentence)
- `sequence_length`: Number of tokens including [CLS] and [SEP]
- `hidden_size`: 768 for bert-base

In [7]:
def get_embeddings(text):
    """Get BERT embeddings for all tokens in text."""
    inputs = tokenizer(text, return_tensors='pt')
    
    with torch.no_grad():  # No gradient computation needed for inference
        outputs = model(**inputs)
    
    # last_hidden_state: [batch, seq_len, 768]
    embeddings = outputs.last_hidden_state
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    return tokens, embeddings[0]  # Return tokens and embeddings for first (only) batch item


def get_word_embedding(text, target_word):
    """Get embedding for a specific word in the text."""
    tokens, embeddings = get_embeddings(text)
    
    # Find the target word in tokens (handle subwords by looking for exact match)
    target_lower = target_word.lower()
    for i, token in enumerate(tokens):
        if token == target_lower:
            return embeddings[i]
    
    # If not found as exact token, might be a subword - find starting token
    for i, token in enumerate(tokens):
        if token.startswith(target_lower[:3]):  # Partial match for subwords
            return embeddings[i]
    
    return None

In [8]:
# Get embeddings for a sentence
text = "The quick brown fox jumps over the lazy dog."
tokens, embeddings = get_embeddings(text)

print(f"Tokens: {tokens}")
print(f"Embedding shape: {embeddings.shape}")  # [seq_len, 768]
print(f"\nFirst token '[CLS]' embedding (first 10 dims):")
print(embeddings[0][:10])

Tokens: ['[CLS]', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '[SEP]']
Embedding shape: torch.Size([12, 768])

First token '[CLS]' embedding (first 10 dims):
tensor([-0.3608,  0.2271, -0.3030, -0.1880,  0.0475, -0.3690,  0.1099,  0.3495,
        -0.1674, -0.2070])


### Contextual Embeddings Demo

The magic of BERT: **same word, different contexts → different embeddings**

Let's compare "bank" in:
1. "I deposited money at the bank" (financial institution)
2. "I sat by the river bank" (edge of water)

In [9]:
# Two sentences with "bank" in different contexts
sent1 = "I deposited money at the bank"
sent2 = "I sat by the river bank"

# Get embeddings for "bank" in each context
bank_financial = get_word_embedding(sent1, "bank")
bank_river = get_word_embedding(sent2, "bank")

# Compute cosine similarity between the two "bank" embeddings
# cosine() returns distance (0 = identical), so similarity = 1 - distance
similarity = 1 - cosine(bank_financial.numpy(), bank_river.numpy())

print(f"Sentence 1: '{sent1}'")
print(f"Sentence 2: '{sent2}'")
print(f"\nCosine similarity between 'bank' embeddings: {similarity:.4f}")
print("\n→ Different contexts produce different embeddings for the same word!")

Sentence 1: 'I deposited money at the bank'
Sentence 2: 'I sat by the river bank'

Cosine similarity between 'bank' embeddings: 0.7393

→ Different contexts produce different embeddings for the same word!


In [10]:
# More examples of contextual embeddings
word_contexts = [
    ("apple", "I ate a delicious apple", "Apple released a new iPhone"),
    ("bat", "The bat flew out of the cave", "He swung the bat and hit a home run"),
    ("cell", "The prisoner was locked in a cell", "The cell divides during mitosis"),
    ("python", "A python is a large snake", "I wrote the code in Python"),
]

print("Cosine similarity for same word in different contexts:\n")
for word, sent1, sent2 in word_contexts:
    emb1 = get_word_embedding(sent1, word)
    emb2 = get_word_embedding(sent2, word)
    
    if emb1 is not None and emb2 is not None:
        sim = 1 - cosine(emb1.numpy(), emb2.numpy())
        print(f"'{word}': {sim:.4f}")
        print(f"  • {sent1}")
        print(f"  • {sent2}\n")

Cosine similarity for same word in different contexts:

'apple': 0.1962
  • I ate a delicious apple
  • Apple released a new iPhone

'bat': 0.4594
  • The bat flew out of the cave
  • He swung the bat and hit a home run

'cell': 0.3688
  • The prisoner was locked in a cell
  • The cell divides during mitosis

'python': 0.4948
  • A python is a large snake
  • I wrote the code in Python



## 4. Sentence Embeddings

For many tasks, we need a single vector representing the entire sentence. Two common approaches:

1. **[CLS] token embedding**: The first token is designed to aggregate sequence information
2. **Mean pooling**: Average all token embeddings (often works better)

For production use, **Sentence-Transformers** provides models fine-tuned specifically for sentence similarity.

In [11]:
def get_sentence_embedding_cls(text):
    """Get sentence embedding using [CLS] token."""
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] is the first token (index 0)
    return outputs.last_hidden_state[0, 0, :]


def get_sentence_embedding_mean(text):
    """Get sentence embedding using mean pooling."""
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    # Average all token embeddings (excluding [CLS] and [SEP] for cleaner results)
    embeddings = outputs.last_hidden_state[0, 1:-1, :]  # Skip first and last tokens
    return embeddings.mean(dim=0)

In [12]:
# Compare the two methods
sentences = [
    "The weather is beautiful today.",
    "It's a lovely sunny day.",
    "I need to buy groceries.",
]

print("Sentence similarity using [CLS] vs Mean pooling:\n")

# CLS method
cls_embs = [get_sentence_embedding_cls(s) for s in sentences]
print("[CLS] token method:")
for i in range(len(sentences)):
    for j in range(i + 1, len(sentences)):
        sim = 1 - cosine(cls_embs[i].numpy(), cls_embs[j].numpy())
        print(f"  '{sentences[i][:30]}...' <-> '{sentences[j][:30]}...': {sim:.4f}")

# Mean pooling method
print("\nMean pooling method:")
mean_embs = [get_sentence_embedding_mean(s) for s in sentences]
for i in range(len(sentences)):
    for j in range(i + 1, len(sentences)):
        sim = 1 - cosine(mean_embs[i].numpy(), mean_embs[j].numpy())
        print(f"  '{sentences[i][:30]}...' <-> '{sentences[j][:30]}...': {sim:.4f}")

Sentence similarity using [CLS] vs Mean pooling:

[CLS] token method:
  'The weather is beautiful today...' <-> 'It's a lovely sunny day....': 0.9552
  'The weather is beautiful today...' <-> 'I need to buy groceries....': 0.8398
  'It's a lovely sunny day....' <-> 'I need to buy groceries....': 0.8608

Mean pooling method:
  'The weather is beautiful today...' <-> 'It's a lovely sunny day....': 0.8432
  'The weather is beautiful today...' <-> 'I need to buy groceries....': 0.5501
  'It's a lovely sunny day....' <-> 'I need to buy groceries....': 0.5491


### Sentence-Transformers (Recommended)

For sentence similarity tasks, use **Sentence-Transformers** — BERT models fine-tuned specifically for semantic similarity. Much better results than raw BERT!

In [13]:
from sentence_transformers import SentenceTransformer

# Load a sentence-transformer model (all-MiniLM-L6-v2 is fast and good)
st_model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode sentences - super simple API!
sentences = [
    "The weather is beautiful today.",
    "It's a lovely sunny day.",
    "I need to buy groceries.",
]

embeddings = st_model.encode(sentences)
print(f"Embedding shape: {embeddings.shape}")  # [num_sentences, 384]

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1281.62it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding shape: (3, 384)


In [14]:
# Compute similarities using sentence-transformers
from sentence_transformers import util

# Compute cosine similarity matrix
cos_sim = util.cos_sim(embeddings, embeddings)

print("Sentence similarity matrix (Sentence-Transformers):\n")
for i, sent in enumerate(sentences):
    print(f"{i}: {sent[:40]}")

print(f"\n{cos_sim}")

Sentence similarity matrix (Sentence-Transformers):

0: The weather is beautiful today.
1: It's a lovely sunny day.
2: I need to buy groceries.

tensor([[1.0000, 0.7101, 0.0845],
        [0.7101, 1.0000, 0.0708],
        [0.0845, 0.0708, 1.0000]])


## 5. Semantic Search

A practical application: find the most similar documents to a query. Unlike keyword search (TF-IDF), semantic search understands meaning — "automobile" matches "car".

In [15]:
# Document corpus
documents = [
    "The cat sits on the windowsill watching birds.",
    "Machine learning models require large datasets.",
    "Python is a popular programming language.",
    "The dog runs in the park chasing squirrels.",
    "Deep learning uses neural networks with many layers.",
    "JavaScript is used for web development.",
    "The kitten plays with a ball of yarn.",
    "Natural language processing analyzes text data.",
]

# Encode all documents
doc_embeddings = st_model.encode(documents)

def semantic_search(query, top_k=3):
    """Find most similar documents to query."""
    query_embedding = st_model.encode([query])
    similarities = util.cos_sim(query_embedding, doc_embeddings)[0]
    
    # Get top-k indices
    top_indices = similarities.argsort(descending=True)[:top_k]
    
    print(f"Query: '{query}'\n")
    print("Top matches:")
    for i, idx in enumerate(top_indices):
        print(f"  {i+1}. [{similarities[idx]:.4f}] {documents[idx]}")

In [16]:
# Try different queries - notice semantic understanding!
semantic_search("feline animals")  # Should match cat/kitten docs

Query: 'feline animals'

Top matches:
  1. [0.3616] The kitten plays with a ball of yarn.
  2. [0.3593] The cat sits on the windowsill watching birds.
  3. [0.2148] The dog runs in the park chasing squirrels.


In [17]:
semantic_search("artificial intelligence")  # Should match ML/DL docs

Query: 'artificial intelligence'

Top matches:
  1. [0.3811] Deep learning uses neural networks with many layers.
  2. [0.3129] Machine learning models require large datasets.
  3. [0.3117] Python is a popular programming language.


In [18]:
semantic_search("coding languages")  # Should match Python/JavaScript docs

Query: 'coding languages'

Top matches:
  1. [0.5095] Python is a popular programming language.
  2. [0.3871] JavaScript is used for web development.
  3. [0.2497] Deep learning uses neural networks with many layers.


---

## Summary: Word2Vec vs BERT

| Aspect | Word2Vec | BERT |
|--------|----------|------|
| **Embedding type** | Static (one vector per word) | Contextual (different vectors based on context) |
| **Architecture** | Shallow neural network | Deep Transformer (12+ layers) |
| **Training objective** | Predict context/target words | Masked language modeling + next sentence |
| **Handles polysemy?** | No — "bank" is always same vector | Yes — "river bank" ≠ "bank account" |
| **Unknown words** | Out-of-vocabulary (OOV) problem | WordPiece handles any word |
| **Dimensions** | Typically 100-300 | 768 (base) or 1024 (large) |
| **Speed** | Very fast | Slower (more computation) |
| **Pre-training data** | Often domain-specific | Massive web corpus (Wikipedia, books) |
| **Best for** | Simple similarity, limited compute | Complex NLU, semantic understanding |

**The embedding evolution:**
1. **TF-IDF/BoW**: Sparse, word-independent
2. **Word2Vec**: Dense, but static
3. **BERT**: Dense and contextual — the current standard

**Next steps:**
- Try different BERT variants (RoBERTa, DistilBERT, ALBERT)
- Fine-tune BERT for specific tasks (classification, NER, QA)
- Explore newer models (GPT, T5, LLaMA)

## Bonus: Visualizing Embeddings

Reduce 768 dimensions to 2D to see how sentences cluster by meaning.

In [19]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Sentences grouped by topic
sentences = [
    # Animals
    "The cat sleeps on the couch.",
    "A dog plays in the yard.",
    "The kitten chases a mouse.",
    # Programming
    "Python is great for data science.",
    "JavaScript powers the web.",
    "Rust is a systems language.",
    # Weather
    "It's raining outside today.",
    "The sun is shining brightly.",
    "Snow covers the ground.",
]

labels = ["animal"] * 3 + ["programming"] * 3 + ["weather"] * 3
colors = {"animal": "red", "programming": "blue", "weather": "green"}

# Get embeddings
embeddings = st_model.encode(sentences)

# Reduce to 2D with PCA
pca = PCA(n_components=2)
reduced = pca.fit_transform(embeddings)

# Plot
plt.figure(figsize=(10, 6))
for i, (x, y) in enumerate(reduced):
    plt.scatter(x, y, c=colors[labels[i]], s=100)
    plt.annotate(sentences[i][:25] + "...", (x, y), fontsize=8, alpha=0.7)

plt.title("BERT Sentence Embeddings (PCA)")
plt.xlabel("PC1")
plt.ylabel("PC2")

# Legend
for label, color in colors.items():
    plt.scatter([], [], c=color, label=label)
plt.legend()
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'